In [1]:
from torchvision.models import *
import wandb
from sklearn.model_selection import train_test_split
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import *
import torch,torchvision
from tqdm import tqdm
device = 'cuda'
PROJECT_NAME = 'Intel-Image-Classification-V3-TL'

In [2]:
def load_data():
    data = []
    labels = {}
    labels_r = {}
    idx = 0
    for label in os.listdir('./data/'):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label
    for folder in os.listdir('./data/'):
        for file in os.listdir(f'./data/{folder}/'):
            img = cv2.imread(f'./data/{folder}/{file}')
            img = cv2.resize(img,(56,56))
            img = img / 255.0
            data.append([
                img,
                np.eye(labels[folder],len(labels))[labels[folder]-1]
            ])
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(d[1])
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)
    X_train = torch.from_numpy(np.array(X_train)).to(device).view(-1,3,56,56).float()
    y_train = torch.from_numpy(np.array(y_train)).to(device).float()
    X_test = torch.from_numpy(np.array(X_test)).to(device).view(-1,3,56,56).float()
    y_test = torch.from_numpy(np.array(y_test)).to(device).float()
    return X,y,X_train,X_test,y_train,y_test,labels,labels_r,idx,data

In [3]:
X,y,X_train,X_test,y_train,y_test,labels,labels_r,idx,data = load_data()

In [4]:
torch.save(X_train,'X_train.pt')
torch.save(y_train,'y_train.pt')
torch.save(X_test,'X_test.pt')
torch.save(y_test,'y_test.pt')
torch.save(labels_r,'labels_r.pt')
torch.save(labels,'labels.pt')
torch.save(X_train,'X_train.pth')
torch.save(y_train,'y_train.pth')
torch.save(X_test,'X_test.pth')
torch.save(y_test,'y_test.pth')
torch.save(labels_r,'labels_r.pth')
torch.save(labels,'labels.pth')

In [37]:
def pred(model):
    imgs = []
    for img in os.listdir('./test/'):
        imgs.append(cv2.resize(cv2.imread(f'./test/{img}'),(56,56))/255.0)
    preds = torch.squeeze(model(torch.from_numpy(np.array(imgs)).view(-1,3,56,56).float()))
    print(preds)
    for i in tqdm(range(len(imgs))):
        img = imgs[i]
        plt.figure(figsize=(12,6))
        plt.title(labels_r[int(torch.argmax(preds[i]))])
        plt.imshow(img)
        plt.title(labels_r[int(torch.argmax(preds[i]))])
        plt.savefig('./img.png')
        plt.close()

In [6]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [7]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [20]:
model = mnasnet1_0().to(device)

In [21]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [22]:
model.classifier[1] = Linear(1280,len(labels))

In [23]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=6, bias=True)
)

In [24]:
criterion = MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [25]:
epochs = 100

In [26]:
batch_size = 32

In [28]:
torch.cuda.empty_cache()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    torch.cuda.empty_cache()
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        model.to(device)
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':get_loss(model,X_batch,y_batch,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':get_accuracy(model,X_batch,y_batch)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()

100%|██████████| 100/100 [19:21<00:00, 11.61s/it]


Acc,█████▁█▁███▇▁▁▁▁▁███▇▆▇▄▂▁█▃█▄▆█▅▂█▅▂▃▁▁
Loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁█▁▃▁▁▁▁▁▁▁▁▁▁▁
Acc,0.0
Loss,0.13021


In [38]:
pred(model.to('cpu'))

tensor([[-0.0953,  0.0841,  0.3705,  0.3797,  0.1428, -0.0013],
        [-0.1346,  0.0912,  0.4055,  0.3785,  0.1084, -0.0011],
        [-0.1513,  0.1321,  0.2963,  0.4006,  0.2329, -0.0010],
        [-0.1407,  0.0806,  0.3739,  0.4117,  0.1169, -0.0011],
        [-0.0915,  0.0907,  0.3819,  0.3660,  0.1399, -0.0011]],
       grad_fn=<SqueezeBackward0>)


100%|██████████| 5/5 [00:00<00:00,  8.72it/s]
